In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect= True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()

print(f"The most recent date in the data set is {most_recent_date}")

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
values = session.query(measurement.date, measurement.prcp).filter(measurement.date > '2016-08-22').order_by(measurement.date).all()

# Perform a query to retrieve the data and precipitation scores
for value in values:
    print(values)

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
values_df = pd.DataFrame(values)
final_values_df = values_df.set_index("date")
final_values_df.head(5)

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
final_values_df.plot()
plt.ylim(0,7)
plt.xticks(rotation=90)
plt.title('Precipitation Data of 12 months')
plt.xlabel('Date')
plt.ylabel('Inches')
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
final_values_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
total_number_of_stations = session.query(station.station).count()
total_number_of_stations

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
most_active_stations = session.query(measurement.station,func.count(measurement.station)).order_by(func.count(measurement.station).desc()).group_by(measurement.station).all() 
most_active_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = session.query(measurement.station,func.count(measurement.station)).order_by(func.count(measurement.station).desc()).group_by(measurement.station).first()

# Highlight the station number
most_active_station_number = most_active_station[0]

# Lowest temperature
lowest_temperature = session.query(func.min(measurement.tobs)).filter(measurement.station==most_active_station_number).all()

# Highest temperature
highest_temperature = session.query(func.max(measurement.tobs)).filter(measurement.station==most_active_station_number).all()

# Average temperature
average_temperature = session.query(func.avg(measurement.tobs)).filter(measurement.station==most_active_station_number).all()

print(f"The most active station number is {most_active_station_number}")
print(f"The Lowest temperature is {lowest_temperature}")
print(f"The Highest temperature is {highest_temperature}")
print(f"The Average temperature is {average_temperature}")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station
most_recent_date = session.query(measurement.date,measurement.tobs).filter(measurement.station == most_active_station_number).\
    order_by(measurement.date.desc()).first()

print(f"The most recent date is {most_recent_date}")

twelve_month_values = session.query(measurement.date,measurement.tobs).filter(measurement.station == most_active_station_number).\
    filter(measurement.date > '2016-08-17').all()

In [ ]:
# Plot the results as a histogram
twelve_month_tobs_values = pd.DataFrame(twelve_month_values)["tobs"]

twelve_month_tobs_values.hist(bins=12)
plt.ylim(0,70)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('TOBS Data of 12 months')
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.show()

# Close session

In [ ]:
# Close Session
session.close()